# Introduction

NOTE: Notebook retained for ... posterity, sorta. When re-running, towards the end there are values for the maximum and minimum temperatures seen in a given map. These values are incorrect, they're a product of a change I had to made. It's not worth the time to rerun the notebook now. See also the sister version of the notebook with the masks.

I had one goal that morphed into another while making this notebook. Initially, I wanted to demonstrate:
- Using the `Instrument` object
- Using PySM3
- A PySM3-centered executor
- A Matplotlib-centered executor

But an issue came up outside this. I wanted to compare what I get from PySM3 foreground simulations with the actual Planck maps. As I went, I thought it may be useful to present this as a "how-I-work" notebook.

For reference, about 3 hours went into making the simulated maps and the comparisons with Planck. That was fast! However, another 10 went into processing figures, rewriting code, and debugging. Then another 8 went into all the verbage, explanation, and running the notebook a few times to get it prepared for use as a demonstration. This is certainly more time than it normally would have been; I was trying to maintain a utility to this notebook as a reference. Hopefully it's worthwhile; if nothing else, it will be easier for me to revisit later.

I've cleared out the figures produced because of the large volume of space they take up (28.3 MB with images; 81.9kB without).

A word of caution: the notebook will generate a Dataset folder with ~2.7 GB of data. An additional 12.4 GB of space is taken for the external science assets, but those are used for many other things by CMB-ML.

## Contents

- [Set-up](#set-up) the notebook
- [The `Instrument` object](#the-instrument-object)
- [PySM3 to produce simulations](#pysm3-to-produce-simulations)
    - [LFI Instrument](#lfi-instrument) including details about each step
    - [PySM3](#pysm3) details about PySM3, and using it to make LFI sims
    - [HFI Instrument](#hfi-instrument) revises the LFI code
- [Building Visualizations](#compare-foregrounds-and-official-planck-maps)
    - [Comparing Maps](#comparing-maps) generating reconstruction and residual maps for comparison
    - [Plotting Version 1](#plotting-version-1) a simple view
    - [Plotting Version 2](#plotting-version-2) a view that clips low and high values
    - [Plotting Version 3](#plotting-version-3) a view that uses Symmetric Log scaling
- [Polishing Display](#cleaning-up-code)
    - Cleaning up the code
    - [Iterating display parameters](#finding-display-parameters)
    - [Histograms](#histograms) were helpful, perhaps they should have been an earlier step for finding display settings
- [Final Plotting](#final-plotting)
    - Cleaning up code again, making maps as subfigures

This notebook isn't nearly as polished as some of the other tutorials. I hope it gives good insight into how I work.

There are a lot of figures generated by the notebook, which bloats the size of this. I've added a flag to disable that output for the version that goes in the main repository. Change this flag when you run the code to see everything.

In [1]:
show_figures = False  # 28.3 MB with figures

# Set-up

There's nothing terribly interesting in the setup. The final cell in this section uses Hydra overrides.

In [2]:
# Ignore this cell. 
# It's needed for the notebook to work, not something to learn.

import sys
import os

# Set the local_system
os.environ["CMB_ML_LOCAL_SYSTEM"] = "generic_lab"

# Add the path to the parent directory so I can import cmb-ml
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, repo_root)

In [3]:
import logging

In [4]:
logger = logging.getLogger("F_Tutorial")
logger.setLevel(logging.DEBUG)

# Outside of a notebook, Hydra will handle the logging. 
handler = logging.StreamHandler()  # StreamHandler sends logs to sys.stdout by default
handler.setLevel(logging.DEBUG)
logger.addHandler(handler)

Note the overrides in the following cell. I set this up initially for a demonstration of the Instrument object. I will use overrides throughout this notebook.

The overrides allow me to make variations on the configuration. Throughout most of the notebook, I'll be producing high-resolution maps. However, at first, I'll use low resolution and a smaller subset of the detectors.

In [5]:
from omegaconf import OmegaConf
from hydra.core.hydra_config import HydraConfig
from hydra import compose, initialize

# There are better ways to do this outside of notebooks.
with initialize(version_base=None, config_path="../cfg"):
    cfg = compose(config_name="config_demoH_instrument", 
                  return_hydra_config=True,
                  overrides=["nside=32",
                             "detectors=[30,70,143,353,857]",
                             "pipeline.raw.assets_out.deltabandpass.path_template=${detector_info_path}"])
    HydraConfig.instance().set_config(cfg)

# The `Instrument` object

The `Instrument` object maintains the parameters that describe the various detectors. I create it using a table containing small amount of information for each.

In [6]:
from cmbml.core import Asset, Namer
from cmbml.utils import Instrument, make_instrument
from cmbml.core.asset_handlers import QTableHandler

In order to build the instrument, I load the table. While it can be done manually, I'm going to use the CMB-ML classes to do this. First I define a `Namer` for the configuration and an `Asset` for the table with detector information.

In [7]:
name_tracker = Namer(cfg)
in_det_table: Asset = Asset(cfg, 
                            source_stage="raw",
                            asset_name="deltabandpass",
                            name_tracker=name_tracker,
                            in_or_out="in")
detector_info = in_det_table.read()

A helper function makes the `Instrument`. The object is frozen Dataclass since, once loaded, nothing needs to change. This renders it immutable, so that it can be used in thread-safe processes.

In [8]:
instrument = make_instrument(cfg, 
                             det_info=detector_info)

The Instrument contains a dictionary of Detector objects (Detector is also a frozen Dataclass).

In [9]:
instrument.dets

{30: Detector(nom_freq=30, fields='I', cen_freq=<Quantity 28.4 GHz>, fwhm=<Quantity 75. arcmin>),
 70: Detector(nom_freq=70, fields='I', cen_freq=<Quantity 70.4 GHz>, fwhm=<Quantity 55. arcmin>),
 143: Detector(nom_freq=143, fields='I', cen_freq=<Quantity 142.876 GHz>, fwhm=<Quantity 32.4 arcmin>),
 353: Detector(nom_freq=353, fields='I', cen_freq=<Quantity 357.5 GHz>, fwhm=<Quantity 22. arcmin>),
 857: Detector(nom_freq=857, fields='I', cen_freq=<Quantity 866.8 GHz>, fwhm=<Quantity 20.6 arcmin>)}

Note that the key for the instrument's dictionary (`freq` in the following cell) is the same as the nominal frequency for flexibility.

In [10]:
print("Frequency, Fields, Center Frequency,    FHWM")
for freq, det in instrument.dets.items():
    print(f"{freq:>3} / {det.nom_freq:>3}: {det.fields:>6}, {det.cen_freq:>12.02f}, {det.fwhm:.1f}")
del freq, det

Frequency, Fields, Center Frequency,    FHWM
 30 /  30:      I,        28.40 GHz, 75.0 arcmin
 70 /  70:      I,        70.40 GHz, 55.0 arcmin
143 / 143:      I,       142.88 GHz, 32.4 arcmin
353 / 353:      I,       357.50 GHz, 22.0 arcmin
857 / 857:      I,       866.80 GHz, 20.6 arcmin


You may notice that the scenario YAML has more detectors than I'm using. This "Full Instrument" is set up to constrain what options are available in the external science sources.

In [11]:
print(OmegaConf.to_yaml(cfg.scenario.full_instrument))

30: IQU
44: IQU
70: IQU
100: IQU
143: IQU
217: IQU
353: IQU
545: I
857: I



This is forward looking, to polarization data being used. Right now, I have just the temperature ("I" for "Intensity") field. However, if I were to use polarization fields "IQU", I would hit a barrier because the 545 and 857 GHz official releases have no polarization data. Were I to create an instrument with "IQU" and the 30 GHz, 143 GHz and 857 GHz detector, it would look like:
```text
Frequency, Fields, Center Frequency,    FHWM
 30 /  30:    IQU,        28.40 GHz, 33.1 arcmin
143 / 143:    IQU,       142.88 GHz, 7.3 arcmin
857 / 857:      I,       866.80 GHz, 4.6 arcmin
```
This way, I would have the ability to create simulations or perform other processes on across all detectors, and more easily look up the fields available. This is not a concern right now, as I have the following setting:

In [12]:
cfg.scenario.map_fields

'I'

I keep track of the detector's raw information using the definitions in a table from the PySM3 release. In the future, I would love to get more precise information from the intrument RIMOs.

In [13]:
print(detector_info)

band center_frequency  fwhm 
           GHz        arcmin
---- ---------------- ------
  30             28.4   75.0
  44             44.1   65.0
  70             70.4   55.0
 100           100.89   43.0
 143          142.876   32.4
 217          221.156   22.3
 353            357.5   22.0
 545            555.2   21.5
 857            866.8   20.6


To prevent issues, I'll be deleting things from the namespace through this notebook. It helps me ensure that I'm not using old data - as long as I've properly deleted variables. <!-- This is a pitfall of jupyter notebooks, and part of the reason I prefer to work in modules. -->

In [14]:
try:
    del name_tracker, instrument
except NameError:  # In case I run this cell multiple times
    pass

# PySM3 to produce simulations

This is where I diverged from my original plan. Instead, in this notebook, I'll be creating foregrounds at high resolution.

## LFI Instrument

In [15]:
import numpy as np
import pysm3

from tqdm import tqdm
from cmbml.core.asset_handlers import HealpyMap

The LFI instrument used a type of detector that led to lower resolution, up to $\textrm{N}_\mathrm{side}=1024$. I process them separately from the HFI detectors.

Note that I have two $\textrm{N}_\mathrm{side}$'s. The first is the output resolution for the maps that I want to produce. The other is the resolution at which PySM3 loads and processes the maps. This enables PySM3 to downgrade the maps with fewer artifacts. The rule, as far as I'm aware, is (for most inputs) to use 2048 whenever the final resolution is 1024 or lower, and double the $\textrm{N}_\mathrm{side}$ when it is higher. Refer to [the PySM3 documentation](https://pysm3.readthedocs.io/en/latest/index.html) for more information.

In [16]:
with initialize(version_base=None, config_path="../cfg"):
    cfg = compose(config_name="config_demoH_instrument", 
                  return_hydra_config=True,
                  overrides=["nside=1024",
                             "detectors=[30,44,70]",
                             "nside_sky=2048",
                             "pipeline.raw.assets_out.deltabandpass.path_template=${detector_info_path}"])
    HydraConfig.instance().set_config(cfg)

It's critical to have the `name_tracker` defined **after** the `cfg` and **before** the `Asset`s, and not again. It's a difficulty that comes from working in notebooks without Executors (which handle this). In the rest of the notebook, that's why I have these elements in the same cell.

In [17]:
name_tracker = Namer(cfg)
instrument = make_instrument(cfg, det_info=detector_info)
out_sky_map = Asset(cfg,
                    source_stage="sim",
                    asset_name="sky_map",
                    name_tracker=name_tracker,
                    in_or_out="out")
out_sky_map_handler: HealpyMap

I wanted to test that I could write a map. Instead of generating a large map and finding out an issue during a long process, I often save tiny,  $\textrm{N}_\mathrm{side}=1$ maps as an initial check. I've left that debugging check in the following cell:

In [18]:
# dummy_map = np.arange(12)
# context = dict(
#     freq=123,
#     sim=0,
#     stage="Simulation"
# )
# with name_tracker.set_contexts(context):
#     out_sky_map.write(data=dummy_map)

## PySM3

The [PySM3 library](https://github.com/galsci/pysm) works by building a model for each component (foregrounds and CMB) and keeping a list of them, in the `Sky` object. Each foreground is a `Component` object. More information can be found in [their documentation](https://pysm3.readthedocs.io/en/latest/).

Two lists are passed to the Sky object at initialization:
- `component_objects` - actual Component classes
- `preset_strings` - shorthand strings

The `preset_strings` are [defined here](https://github.com/galsci/pysm/blob/f9d1f1abc72af8bc3bd7cd7c398b16106d32d2bb/src/pysm3/data/presets.cfg#L1) (link to GitHub) and [described here](https://pysm3.readthedocs.io/en/latest/models.html#models).

The `Sky` object first builds from the `component_objects` list. In the [actual simulation code](../cmbml/sims/stage_executors/H_make_observations.py), I create a placeholder `Component` and swap it out once I've generated a CMB. That swap lets me run the simulation more quickly, and I'm 80% confident that it won't cause issues.

In the following cell, I create a `Sky` with the preset strings defined in [the configuration](../cfg/config_demoH_instrument.yaml#L42).

In [19]:
nside_sky = cfg.model.sim.nside_sky
preset_strings = list(cfg.model.sim.preset_strings)
output_unit = cfg.scenario.units
sky = pysm3.Sky(nside=nside_sky,
                preset_strings=preset_strings,
                output_unit=output_unit)  # ~ 60 seconds

As mentioned before, I only use a few detectors for now (the LFI detectors). I'll check that those are the only ones in my instrument.

In [20]:
for det in instrument.dets:
    print(instrument.dets[det])

Detector(nom_freq=30, fields='I', cen_freq=<Quantity 28.4 GHz>, fwhm=<Quantity 75. arcmin>)
Detector(nom_freq=44, fields='I', cen_freq=<Quantity 44.1 GHz>, fwhm=<Quantity 65. arcmin>)
Detector(nom_freq=70, fields='I', cen_freq=<Quantity 70.4 GHz>, fwhm=<Quantity 55. arcmin>)


Now that I know I've got the correct detectors set up, I can iterate over them and produce the LFI simulations.

In [21]:
out_nside = cfg.scenario.nside
for freq, det in tqdm(instrument.dets.items()):  # ~ 90 seconds per frequency
    sky_obs = sky.get_emission(det.cen_freq)
    sky_obs_I = sky_obs[0]  # PySM3 returns I, Q, U; we only want I
    smooth_I = pysm3.apply_smoothing_and_coord_transform(
        sky_obs_I,
        fwhm=det.fwhm,
        output_nside=out_nside
    )

    # Hard-coding for now... this is why Executors are nice ;)
    column_names = ["I_STOKES"]
    context = dict(
        freq=freq,
        sim=0,
        stage="Simulation"
    )
    with name_tracker.set_contexts(context):
        out_sky_map.write(data=smooth_I, column_names=column_names)

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [04:15<00:00, 85.02s/it]


## HFI Instrument

The process for the HFI signals is the same. I've cleaned up the code a bit. I'm not a huge fan of repeating large chunks of code like this, since I often miss small details when doing this. However, this is the work I did to make the notebook. I want to show how I rearranged my code as I worked.

I change my initialization slightly: now I'm targetting a higher resolution final map for the HFI frequencies, so the output is 2048 and the sky is at 4096.

In [22]:
# Same as above, with slight rearrangement of lines to match typical Executor layout

# Get the configuration
with initialize(version_base=None, config_path="../cfg"):
    cfg = compose(config_name="config_demoH_instrument", 
                  return_hydra_config=True,
                  overrides=["nside=2048",
                             "detectors=[100,143,217,353,545,857]",
                             "nside_sky=4096",
                             "pipeline.raw.assets_out.deltabandpass.path_template=${detector_info_path}"])
    HydraConfig.instance().set_config(cfg)

# Initialization
name_tracker = Namer(cfg)
out_sky_map = Asset(cfg,
                    source_stage="sim",
                    asset_name="sky_map",
                    name_tracker=name_tracker,
                    in_or_out="out")
out_sky_map_handler: HealpyMap

nside_sky = cfg.model.sim.nside_sky
preset_strings = list(cfg.model.sim.preset_strings)
output_unit = cfg.scenario.units
out_nside = cfg.scenario.nside

instrument = make_instrument(cfg, det_info=detector_info)

# For .execute() method
sky = pysm3.Sky(nside=nside_sky,
                preset_strings=preset_strings,
                output_unit=output_unit)  # ~ 150 seconds

# For each simulation, iterate over detectors
for freq, det in tqdm(instrument.dets.items()):  # ~ 90 seconds per frequency
    sky_obs = sky.get_emission(det.cen_freq)
    sky_obs_I = sky_obs[0]  # PySM3 always returns IQU maps, save only I
    smooth_I = pysm3.apply_smoothing_and_coord_transform(
        sky_obs_I,
        fwhm=det.fwhm,
        output_nside=out_nside
    )

    column_names = ["I_STOKES"]
    context = dict(
        freq=freq,
        sim=0,
        stage="Simulation"
    )
    with name_tracker.set_contexts(context):
        out_sky_map.write(data=smooth_I, column_names=column_names)

100%|██████████| 6/6 [08:48<00:00, 88.08s/it]


At this point, the intention was to convert this to an Executor. I got derailed while previewing the maps.

# Developing Visualizations

## Setting up for comparison

Now that I have foreground maps, I can compare these to Planck's official release. Note that I no longer care about $\textrm{N}_\text{side}$, but I set up the config to force me to set values. I set them to `None` (`null` in YAML) so I don't make a mistake.

I need to set up `Asset`s for each class of maps. I don't like doing this, but I'm trying to illustrate how I do things as I develop. Hard-coding is another option for development, but I want this to run on any system and I know I'll set up an Executor later.

In [23]:
# Set up for easier access for all simulated frequencies
with initialize(version_base=None, config_path="../cfg"):
    cfg = compose(config_name="config_demoH_instrument", 
                  return_hydra_config=True,
                  overrides=[
                             "nside=null",  # Nside varies, we'll handle that externally
                             "detectors=[30,44,70,100,143,217,353,545,857]",
                             "nside_sky=null",
                             "pipeline.raw.assets_out.deltabandpass.path_template=${detector_info_path}"])
    HydraConfig.instance().set_config(cfg)
name_tracker = Namer(cfg)
instrument = make_instrument(cfg, det_info=detector_info)

# Note that "source_stage" refers to the stage where the Asset was PRODUCED.
planck_obs_asset = Asset(cfg,
                         source_stage="raw",
                         asset_name="noise_src_varmaps",  # TODO: Change this name in config and references
                         name_tracker=name_tracker,
                         in_or_out="in")
planck_pred_asset = Asset(cfg,
                          source_stage="raw",
                          asset_name="mask_src_map",  # TODO: Change this name in config and references
                          name_tracker=name_tracker,
                          in_or_out="in")
sky_map_asset = Asset(
                      cfg,
                      source_stage="sim",
                      asset_name="sky_map",
                      name_tracker=name_tracker,
                      in_or_out="in"  
                      )

I have a couple functions that get file paths for the official Planck Assets. These also download those files if they're missing. I don't need to use the download option.

Note that, if I did, I avoid setting `download=True` until I've confirmed that the location is correct.

In [24]:
from get_data.utils.get_planck_data_ext import get_planck_obs_data_ext, get_planck_pred_data_ext

In [25]:
# Get file paths for the external science assets
obs_paths = {}
pred_path = None
with name_tracker.set_context("filename", "placeholder"):
    obs_asset_dir = planck_obs_asset.path.parent
    for freq in instrument.dets:
        obs_paths[freq] = get_planck_obs_data_ext(detector=freq,
                                                  assets_directory=obs_asset_dir,
                                                  download=False)
    pred_asset_name = planck_pred_asset.path.name
    pred_asset_dir = planck_pred_asset.path.parent
    pred_path = get_planck_pred_data_ext(assets_directory=pred_asset_dir,
                                         download=False)

The LFI frequency maps (30, 44, and 70 GHz) are released at $\textrm{N}_\textrm{side}=1024$, while the HFI maps (100+ GHz) are at $\textrm{N}_\textrm{side}=2048$. I'll downgrade the prediction in alm space using a utility function, `downgrade_by_alm`, for use when comparing to the LFI maps.

In [26]:
from cmbml.utils.physics_downgrade_by_alm import downgrade_by_alm

In [27]:
pred_map_2048 = HealpyMap().read(path=pred_path)  # ~15 s
pred_map_1024 = downgrade_by_alm(pred_map_2048, target_nside=1024)  # ~20 s

## Comparing maps

In this subsection, I get the reconstructed map (the forgrounds plus Planck's CMB prediction) and the residual map (the difference between reconstruction and Planck's released observations).

I'll run this for multiple detector frequencies using a for-loop, but while developing, I want to start with one frequency and check results as I go. To do this, I have a cell that takes the place of my `for freq in instrument.dets:` that just sets to the initial detector frequency (hard-coding is ok here, I think). Then I have all the substeps broken out:
- load observation maps
- set the prediction map depending on if the frequency is LFI or HFI
- load sky maps (the simulations of foregrounds)
- create a figure for each

After each of these substeps, I've used to check what I've got and massage the data into the proper form. I note a cell with these kinds of operations below; there were more, but I've tidied this up to be readable.

In [28]:
# Instead of "for freq in instrument.dets:"
freq = 30

In [29]:
# Get Planck official observation map
obs_map = HealpyMap().read(path=obs_paths[freq])
obs_map = obs_map.to("uK_CMB")
obs_map = obs_map[0]  # Just the I map

In [30]:
# Checking different aspects of the loaded map. Each line would be run separately
# obs_map.shape   # Confirming shape (which led to the line "obs_map = obs_map[0]" above)
# obs_map.value   # I realized that this is an AstroPy Quantity, which I can make use of
# obs_map.unit    # Confirming units (which led to the line "obs_map = obs_map.to("uK_CMB")" above)

In [31]:
# Get the appropriate prediction map
if freq in [30, 44, 70]:
    pred_map = pred_map_1024
    # pred_map = pred_map  # downgrade_by_alm already made it just the 'I' map
else:
    pred_map = pred_map_2048  # OOPS! Bug here! I'll note and fix it later, this is FYI
    # pred_map = pred_map_2048[0]  # Just the I map (this is the fix for the previous line)
pred_map = pred_map.to("uK_CMB")

# Checking, similar to above, would occur in a subsequent (removed) cell

In [32]:
# Get the sky map
context = dict(sim=0, freq=freq, stage="Simulation")
with name_tracker.set_contexts(context):
    sky_map = sky_map_asset.read()
sky_map = sky_map.to("uK_CMB")
sky_map = sky_map[0]  # Just the I map

# Checking, similar to above, would occur in a subsequent (removed) cell

In [33]:
# And last, get the "reconstructed" map and the residual 
# between Planck's observation and the reconstruction
recon_map = pred_map + sky_map
residual = obs_map - recon_map

I'm done getting the maps. Now I want to take a look.

(Later, I'll revise this and switch to `recon_map` - `obs_map`, so that where I'm generating stuff that's too hot, it shows up as too hot in the figure.)

## Plotting, Version 1

I need to view the results. I use Healpy and Matplotlib to do so.

In [34]:
import healpy as hp
import matplotlib.pyplot as plt

I could (and did) view the maps within the notebook. I've commented out those lines to remove the figures. If I left them in, the notebook would be bloated. I strongly suggest uncommenting the cells and running the notebook!

In [35]:
if show_figures:
    hp.mollview(obs_map, unit=obs_map.unit, title=f"Planck Observed Map, {freq} GHz")

# Map displayed has range of ~-2000 to 200,000 uK_CMB; 
# speckles along the Galactic plane, and a couple point sources visible.
# Most of the map is a single shade indicating values near 0.

In [36]:
if show_figures:
    hp.mollview(recon_map, unit=recon_map.unit, title=f"Reconstructed Map, {freq} GHz")

# Map displayed has range of ~ -400 to 200,000 uK_CMB; 
# speckles along the Galactic plane, and a couple point sources visible.
# Most of the map is a single shade indicating values near 0.
# There seem to be more point sources, and the map seems to be ... blurrier

In [37]:
if show_figures:
    hp.mollview(residual, unit=residual.unit, title=f"Residual Map, {freq} GHz")

# Map displayed has range of ~-200,000 to 100,000 uK_CMB; 
# speckles along the Galactic plane, and a couple point sources visible. 
# Most of the map is a single shade indicating value near 0

The initial results are disappointing. I'll produce better plots.

## Plotting, Version 2

There's a lot of lost information because of the wide value range. I need to constrain it. I don't know the right values, so I hard-code some tests for this particular frequency. I'll make the hard-coding into parameters and put them into the config file later.

In [38]:
from cmbml.utils.planck_cmap import colombi1_cmap

display_params = dict(min=-1000, max=1000, cmap=colombi1_cmap)

In [39]:
if show_figures:
    hp.mollview(obs_map, unit=obs_map.unit, title=f"Planck Observed Map, {freq} GHz", **display_params)

# Map looks more distinct now.

In [40]:
if show_figures:
    hp.mollview(recon_map, unit=recon_map.unit, title=f"Reconstructed Map, {freq} GHz", **display_params)

# Comparing the map to the previous one, it seems like the hot regions are hotter (larger).

In [41]:
if show_figures:
    hp.mollview(residual, unit=residual.unit, title=f"Residual (obs-recon) Map, {freq} GHz", **display_params)

# The map reinforces previous observation... 
# it seems as though there may be point sources in the reconstruction
# which are larger than in the observed map (the residual has small regions 
# of high values surrounded by low values)

These are better, but I don't like that I can't visualize the extremes well. I've got another idea.

## Plotting, Version 3

I'm going to try one more tool that's a bit more fiddly - plotting with a symmetric log-scale colorbar. Since the code is long, I've create a helper module that does this for me. When working with an external module and actively developing it in a notebook, I'll use `importlib` so I can reload it with changes, as is done in the following cell.

In [42]:
import importlib

from cmbml.utils import cmap_symlog

importlib.reload(cmap_symlog)

from cmbml.utils.cmap_symlog import SymLogPlotSettings, single_symlog_map_fig

That module sets up a `dataclass` so I can define and input parameters more simply. The following is the instance of the dataclass that I'll use. Since this is getting super-fiddly, I hard-code values here while doing development work. I'm going to tune them for the 30 GHz detector first (continuing the work from above).

In [43]:
# Define parameters for Symmetric Logarithmic scaling
symlog_settings = SymLogPlotSettings(
    # vmin=-1e4,  # Values used in explanation below
    # vmax=1e8,
    # linthresh=1e3,
    # linscale=2,
    vmin=-5000,
    vmax=5000,
    linthresh=500,
    linscale=2,
    unit=obs_map.unit,
    ticks=[-5000, -1000, -500, 0, 500, 1000, 5000]
)

The [Symmetric Logarithmic scaling](https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.SymLogNorm.html) (link to Matplotlib documentation) defines a linear threshold, $t$. Between $\pm t$, colors are mapped linearly. Outside this range, the values are log-scaled. It makes it so that I can "squish" the extreme values outside the threshold.

I've tuned the values of `vmin`, `vmax`, `linthresh`, and `linscale`.
- `vmin`: The lowest value. 
    - Values below this get mapped to it (leading to distortion, though less)
    - This is a little below the CMB temperature
- `vmax`: The highest value.
    - This is a little above the CMB temperature
- `linthresh`: The threshold, $t$
    - Values $\in [-t, t]$ are mapped to the color scale linearly
- `linscale`: The width, in decades, of the linear scale

#### Explanation

If you're having trouble with that, the normalization function mapping the domain of temperature values to a domain of color values.

Let's use these settings for an example, and consider generic units of color-bar width:
- `vmin` ($v_\mathit{min}$) = $-10^4$
- `vmax` ($v_\mathit{max}$) = $10^8$ 
- `linthresh` ($t$) = $10^3$
- `linscale` = $2$

Considering just the positive values:
- There are five decades between $t$ and $v_\mathit{max}$ ($10^3$ to $10^8$); that's 5 units.
- The region from $0$ to $10^3$ ($t$) is set by `linscale`; that's 2 units.

On the negative side:
- There is one decades between $-t$ and $v_\mathit{min}$ ($-10^3$ to $-10^4$); that's 1 unit.
- The region from $0$ to $-10^3$ ($-t$) is set by `linscale`; that's 2 units. (No different from the positive side)

Now the numberline can be drawn out: there are 3 units on the negative side and 7 units on the positive side (total 10). The original colorbar has the "neutral" color at the middle; it is stretched so that it occurs at 30% of the total width. If there were ticks for $\pm t$, they would occur at 10% and 50% of the total width.

#### Resuming

I'm not a huge fan of what I've done in [that module](../cmbml/utils/cmap_symlog.py): returning the actual `Plot` object from Matplotlib. I do so because then I can easily switch between `.show()` and `.savefig()`. There's probably a better solution, but this is a work-in-progress.

Note: when running this, I used `show_figures=True`; I've set it to `False` in this version of the notebook because of the large file size. I hope to put together another repository with these notebooks including images.

In [44]:
if show_figures:
    res_plt = single_symlog_map_fig(obs_map, symlog_settings, title=f"Planck Observed Map, {freq} GHz")
    res_plt.show()
# I can see both the extreme values and the proximal values more clearly now.

In [45]:
if show_figures:
    res_plt = single_symlog_map_fig(recon_map, symlog_settings, title=f"Reconstructed Map, {freq} GHz")
    res_plt.show()
# The higher temperature of the reconstruction is more visible.

In [46]:
if show_figures:
    res_plt = single_symlog_map_fig(residual, symlog_settings, title=f"Difference (obs-recon) Map, {freq} GHz")
    res_plt.show()
# The difference map shows much the same as before

Those maps now look pretty good. I'm now going to clean up my code and make it more compact.

# Polishing Display

## Cleaning up code

Now, I want to investigate: can I find a set of plotting parameters which will be consistent for all the maps? They're hard-coded in a cell up there and I don't like that for the long-run, but it's inconvenient to adjust my top-level config. Here's what I do:
- Consolidate the code 
    - and test it to make sure it's actually the same!
- Make it a method with a parameters for the different frequencies and symlog_settings
    - and test it!
- Iterate on the display settings

**Consolidate the code**: First, I've pulled the contents of the above cells into a single cell. I rerun it to ensure that I get what I think I should get.

In [47]:
# Instead of "for freq in instrument.dets:"
freq = 30

# Get Planck official observation map
obs_map = HealpyMap().read(path=obs_paths[freq])
obs_map = obs_map.to("uK_CMB")
obs_map = obs_map[0]  # Just the I map

# Get the appropriate prediction map
if freq in [30, 44, 70]:
    pred_map = pred_map_1024
    # pred_map = pred_map  # downgrade_by_alm already made it just the 'I' map
else:
    pred_map = pred_map_2048  # Just the I map
pred_map = pred_map.to("uK_CMB")

# Get the sky map
context = dict(sim=0, freq=freq, stage="Simulation")
with name_tracker.set_contexts(context):
    sky_map = sky_map_asset.read()
sky_map = sky_map.to("uK_CMB")
sky_map = sky_map[0]  # Just the I map

recon_map = pred_map + sky_map
residual = obs_map - recon_map

# Define parameters for Symmetric Logarithmic scaling
symlog_settings = SymLogPlotSettings(
    vmin=-5000,
    vmax=5000,
    linthresh=500,
    linscale=2,
    unit=obs_map.unit,
    ticks=[-5000, -1000, -500, 0, 500, 1000, 5000]
)

if show_figures:
    res_plt = single_symlog_map_fig(obs_map, symlog_settings, title=f"Planck Observed Map, {freq} GHz")
    res_plt.show()

    res_plt = single_symlog_map_fig(recon_map, symlog_settings, title=f"Reconstructed Map, {freq} GHz")
    res_plt.show()

    res_plt = single_symlog_map_fig(residual, symlog_settings, title=f"Difference (obs-recon) Map, {freq} GHz")
    res_plt.show()

Those maps are the same. Great!

**Make it into a method** Now, instead of a bunch of lines in the global namespace, I'm going to put it into a single method. I prepend the parameters for the method with "_" so I know I'm not accidentally using external ones. I sometimes even copy this into an empty Python module and use the IDE to let me know what's not in the global namespace.

In [48]:
from pysm3 import units as u
from dataclasses import replace

from cmbml.utils import physics_units

importlib.reload(physics_units)

from cmbml.utils.physics_units import convert_units

In [49]:
def get_maps(_freq):
    obs_map = HealpyMap().read(path=obs_paths[_freq])
    obs_map = obs_map[0]  # Just the I map

    # Get the appropriate prediction map
    if _freq in [30, 44, 70]:
        pred_map = pred_map_1024     # downgrade_by_alm already made it just the 'I' map
    else:
        pred_map = pred_map_2048[0]  # Just the I map

    # Get the sky map
    context = dict(sim=0, freq=_freq, stage="Simulation")
    with name_tracker.set_contexts(context):
        sky_map = sky_map_asset.read()
    sky_map = sky_map[0]  # Just the I map

    # print([x.unit for x in [obs_map, pred_map, sky_map]])

    # Convert 545 and 857 GHz maps to uK_CMB
    cen_freq = instrument.dets[_freq].cen_freq

    # Really, only pred_map is in MJy/sr (for 545, 857 GHz), so I don't need to use the 
    #         convert_units() function for obs_map and sky_map, but they may be in K_CMB or uK_CMB
    pred_map = convert_units(pred_map, u.uK_CMB, cen_freq)
    obs_map = convert_units(obs_map, u.uK_CMB, cen_freq)
    sky_map = convert_units(sky_map, u.uK_CMB, cen_freq)

    recon_map = pred_map + sky_map
    residual = recon_map - obs_map
    # print([x.unit for x in [obs_map, pred_map, sky_map]])
    return obs_map, pred_map, sky_map, recon_map, residual


In [50]:
def plot_differences_orig(_freq, _symlog_settings):
    obs_map, pred_map, sky_map, recon_map, residual = get_maps(_freq)

    # Need to get the units from the existing maps
    # the replace() method is needed because the unit is a property of the map,
    # which wasn't available outside the loop
    replace(_symlog_settings, unit=obs_map.unit)

    res_plt = single_symlog_map_fig(obs_map, _symlog_settings, title=f"Planck Observed Map, {_freq} GHz")
    res_plt.show()

    res_plt = single_symlog_map_fig(recon_map, _symlog_settings, title=f"Reconstructed Map, {_freq} GHz")
    res_plt.show()

    res_plt = single_symlog_map_fig(residual, _symlog_settings, title=f"Difference (obs-recon) Map, {_freq} GHz")
    res_plt.show()

In [51]:
# Define parameters for Symmetric Logarithmic scaling
symlog_settings = SymLogPlotSettings(
    vmin=-1e4,
    vmax=1e4,
    linthresh=4e2,
    linscale=1.8,
    unit=None,
    ticks=[-1e4, -1e3, -4e2, 0, 4e2, 1e3, 1e4]
)

## Finding display parameters

Now I can just call that function for the different frequencies and get the same results. 

In [52]:
if show_figures:
    plot_differences_orig(30, symlog_settings)  # LFI freqs take ~20s, HFI freqs take ~60s
# This confirms that the function works as expected

In [53]:
if show_figures:
    plot_differences_orig(100, symlog_settings)  # LFI freqs take ~20s, HFI freqs take ~60s
# This checked that the function works for HFI frequencies
# It's also a good check for smaller extreme values

In [54]:
if show_figures:
    plot_differences_orig(217, symlog_settings)  # LFI freqs take ~20s, HFI freqs take ~60s
# This checked that the function works for HFI frequencies
# It's also a good check for smaller extreme values

In [55]:
if show_figures:
    plot_differences_orig(353, symlog_settings)  # LFI freqs take ~20s, HFI freqs take ~60s
# This checked that the function works for HFI frequencies
# It's also a good check for smaller extreme values

At 353 GHz, I have trouble resolving details, since it's so "hot." I've tried a different set of parameters below. (I later decided that instead of having three different settings, I'd stick with what's above.)

In [56]:
symlog_settings = SymLogPlotSettings(
    vmin=-5e3,
    vmax=1e6,
    linthresh=5e2,
    linscale=2,
    unit=None,
    ticks=[-5e3, -5e2, 0, 500, 1e4, 1e5, 1e6]
)
if show_figures:
    plot_differences_orig(353, symlog_settings)  # LFI freqs take ~20s, HFI freqs take ~60s
# This has very high values, it made me increase vmax to 1e6

Similarly, as I went to higher frequencies,I encountered the issues again. I've skipped the bad versions here.

In [57]:
symlog_settings = SymLogPlotSettings(
    vmin=-1e7,
    vmax=1e7,
    linthresh=4000,
    linscale=0.1,
    unit=None,
    ticks=[-1e7, -1e6, -1e5, -1e4, 0, 1e4, 1e5, 1e6, 1e7]
)
if show_figures:
    plot_differences_orig(545, symlog_settings)  # LFI freqs take ~20s, HFI freqs take ~60s
# This has very high values, it made me increase vmax to 1e7

In [58]:
if show_figures:
    plot_differences_orig(857, symlog_settings)  # LFI freqs take ~20s, HFI freqs take ~60s
# This has very high values, but it's 

In [59]:
if show_figures:
    plot_differences_orig(353, symlog_settings)  # LFI freqs take ~20s, HFI freqs take ~60s
# This ended up too pale. If I'm trying for just two sets of settings, I need to rethink this.
# It's also taking up a ton of space, so I think I need to ... change my figure to have three subfigs.

As I run it, I found some bugs above:
- Only the LFI frequencies get the downgrade_by_alm() treatment, which has the side effect of producing singular temperature maps. I think that means I should fix downgrade_by_alm(), so it's on my to-do list later. For now, I've got a simple bugfix applied.
- The map units for 545 and 857 GHz detectors are in MJy/sr, but my simulations output uK_CMB; the conversion is non-trivial and needed a special function.

## Histograms

I also found that I had difficulty seeing what was going on for the highest frequencies (357, 545, and 857 GHz). I decided to have a separate set of parameters for viewing those. I also wanted another way to view the data, so I've put together a histogram of intensity values. It's not great for understanding, but it will convey the point that the simulations have higher values than Planck's observations.

In [60]:
obs_map, pred_map, sky_map, recon_map, residual = get_maps(353)  # ~20s
obs_map.min(), obs_map.max(), obs_map.mean(), obs_map.std()
del obs_map, pred_map, sky_map, recon_map, residual

In [61]:
def make_hist_orig(_freq):
    obs_map, pred_map, sky_map, recon_map, residual = get_maps(_freq)  # ~20s
    use_min = 100 if obs_map.min() < 0 else obs_map.min().value
    bins = np.logspace(np.log10(use_min), np.log10(max(obs_map.value)), 20)
    plt.hist(obs_map.value, bins=bins[:], histtype="step", color="tab:orange", label="Observed")
    plt.hist(obs_map.value, bins=bins[:], alpha=0.2, color="tab:orange")
    plt.hist(recon_map.value, bins=bins[:], histtype="step", color="tab:green", label="Reconstructed")
    plt.hist(recon_map.value, bins=bins[:], alpha=0.2, color="tab:green")
    plt.xscale("log")
    plt.yscale("log")
    plt.ylabel("Pixel Count")
    plt.xlabel(f"$\\Delta T $" + f" ({obs_map.unit.to_string('latex')})")
    plt.title(f"Pixel Histograms for Observed and Reconstructed Maps at {_freq} GHz")
    plt.legend()
    plt.show()

In [62]:
if show_figures:
    make_hist_orig(353)  # ~20s

In [63]:
if show_figures:
    make_hist_orig(545)  # ~20s

In [64]:
if show_figures:
    make_hist_orig(857)  # ~20s

Ok - great! Now I can see that the values are a bit strange across the board. The histograms only work for the highest frequencies, when all values are positive (or mostly positive), but I don't really need them for the lower frequency detectors.

# Final Plotting

Now that I've got the method for individual figures hammered out, I'm going to define the methods one last time and rerun them. This lets me know that everything has been applied consistently.

I've also put all the maps into a single figure so it's easier to see them side-by-side.

I couldn't figure out how to get matplotlib and healpy to play nice and give me the image without a huge white border. Instead, here's a function that crops it down to size. (Thanks ChatGPT)

In [65]:
from PIL import Image

def crop_figure(path, margin: int = 10):
    image = Image.open(path)
    initial_size = image.size

    # Convert image to grayscale for thresholding
    gray_image = image.convert("L")
    image_array = np.array(gray_image)

    # Detect background (assume nearly white background, adjust threshold if needed)
    threshold = 250  # Set based on your image's background brightness
    mask = image_array < threshold

    # Find the bounding box of the non-background region
    coords = np.argwhere(mask)
    if coords.size == 0:
        print("No content detected—possible all-white image.")
        return

    # Get bounding box
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0)

    # Expand the bounding box by the specified margin (ensure no negative values)
    left = max(x_min - margin, 0)
    upper = max(y_min - margin, 0)
    right = min(x_max + margin, initial_size[0])
    lower = min(y_max + margin, initial_size[1])

    # Crop and get the final size
    cropped_image = image.crop((left, upper, right, lower))

    # Save the cropped image
    cropped_image.save(path)

I've got the new function for putting together maps in the same outside module as before; I just need to import the new functions.

In [66]:
from matplotlib.gridspec import GridSpec
from cmbml.utils import cmap_symlog

importlib.reload(cmap_symlog)

from cmbml.utils.cmap_symlog import SymLogPlotSettings, many_symlog_map_fig, many_symlog_map_add_cbar

When creating the figures, I now use the outside functionality to produce a single subfigure. I have another method that follows the adding of each map in order to add the colorbar.

There's a strange issue with spacing when putting Healpy maps in the same figure. I remedy this with `Gridspec`. Unfortunately, it puts a wide margin around the figures, but only in the saved version, not in the displayed version of the figure. This is why the `crop_figure()` method is there.

In [67]:
def plot_differences(_freq, _symlog_settings, out_asset=None, show=False, save=True):
    obs_map, pred_map, sky_map, recon_map, residual = get_maps(_freq)

    print(f"Observation: {obs_map.min():.2e} - {obs_map.max():.2e}, "\
          f"Reconstruction: {recon_map.min():.2e} - {recon_map.max():.2e}, "\
          f"Residual: {residual.min():.2e} - {residual.max():.2e}")

    replace(_symlog_settings, unit=obs_map.unit)

    fig = plt.figure(figsize=(30, 10))
    gs = GridSpec(nrows=1, ncols=3, figure=fig, hspace=0.01, wspace=0.01)

    axs = [fig.add_subplot(gs[0, i]) for i in range(3)]

    many_symlog_map_fig(obs_map, _symlog_settings, axs[0], title=f"Planck Observed Map, {_freq} GHz")
    many_symlog_map_fig(recon_map, _symlog_settings, axs[1], title=f"Reconstructed Map, {_freq} GHz")
    many_symlog_map_fig(residual, _symlog_settings, axs[2], title=f"Difference (recon - obs) Map, {_freq} GHz")
    many_symlog_map_add_cbar(_symlog_settings, fig)

    if save and out_asset is not None:
        context = dict(
            freq=_freq,
            sim=0,
            stage="Simulation"
        )
        with name_tracker.set_contexts(context):
            path = out_asset.write()
        plt.savefig(path)
        crop_figure(path)
        print(f"Saved figure to {path}")
    elif save:
        raise ValueError("If saving, an output asset must be provided.")
    if show:
        plt.show()
    plt.close()

I've also added functionality to the `make_hist_orig()` method for saving the figures.

In [68]:
def make_hist(_freq, out_asset=None, show=False, save=True):
    obs_map, pred_map, sky_map, recon_map, residual = get_maps(_freq)  # ~20s
    use_min = 100 if obs_map.min() < 0 else obs_map.min().value
    bins = np.logspace(np.log10(use_min), np.log10(max(obs_map.value)), 20)
    plt.hist(obs_map.value, bins=bins[:], histtype="step", color="tab:orange", label="Observed")
    plt.hist(obs_map.value, bins=bins[:], alpha=0.2, color="tab:orange")
    plt.hist(recon_map.value, bins=bins[:], histtype="step", color="tab:green", label="Reconstructed")
    plt.hist(recon_map.value, bins=bins[:], alpha=0.2, color="tab:green")
    plt.xscale("log")
    plt.yscale("log")
    plt.ylabel("Pixel Count")
    plt.xlabel(f"$\\Delta T $" + f" ({obs_map.unit.to_string('latex')})")
    plt.title(f"Pixel Histograms for Observed and Reconstructed Maps at {_freq} GHz")
    plt.legend()
    if save and out_asset is not None:
        context = dict(
            freq=_freq,
            sim=0,
            stage="Simulation"
        )
        with name_tracker.set_contexts(context):
            path = out_asset.write()
        plt.savefig(path)
        crop_figure(path)
        print(f"Saved figure to {path}")
    if show:
        plt.show()
    plt.close()

Now I reload my configuration to start fresh (checking for issues). Because of difficulties earlier, I ran this for each map individually first. I've left those cells at the bottom as a sort of ... recognition. My preference is to use a for-loop in the final run, to guarantee that everything is run the same way.

In [69]:
# Set up for easier access for all simulated frequencies
with initialize(version_base=None, config_path="../cfg"):
    cfg = compose(config_name="config_demoH_instrument", 
                  return_hydra_config=True,
                  overrides=[
                             "nside=2048",  # Nside varies, we'll handle that externally
                             "detectors=[30,44,70,100,143,217,353,545,857]",
                             "nside_sky=4096",
                             "pipeline.raw.assets_out.deltabandpass.path_template=${detector_info_path}"])
    HydraConfig.instance().set_config(cfg)
name_tracker = Namer(cfg)
instrument = make_instrument(cfg, det_info=detector_info)

print(OmegaConf.to_yaml(cfg.pipeline.compare))

planck_obs_asset = Asset(cfg,
                         source_stage="raw",
                         asset_name="noise_src_varmaps",  # TODO: Change this name in config and references
                         name_tracker=name_tracker,
                         in_or_out="in")
planck_pred_asset = Asset(cfg,
                          source_stage="raw",
                          asset_name="mask_src_map",  # TODO: Change this name in config and references
                          name_tracker=name_tracker,
                          in_or_out="in")
sky_map_asset = Asset(
                      cfg,
                      source_stage="sim",
                      asset_name="sky_map",
                      name_tracker=name_tracker,
                      in_or_out="in"  
                      )
map_fig_asset = Asset(cfg,
                      source_stage="compare",
                      asset_name="fig_maps",
                      name_tracker=name_tracker,
                      in_or_out="out")
map_hist_asset = Asset(cfg,
                      source_stage="compare",
                      asset_name="fig_hist",
                      name_tracker=name_tracker,
                      in_or_out="out")

assets_out:
  fig_maps:
    handler: Figure
    path_template: '{root}/{dataset}/{stage}/{sim}/fig_maps_{freq}.png'
  fig_hist:
    handler: Figure
    path_template: '{root}/{dataset}/{stage}/{sim}/fig_hist_{freq}.png'
assets_in:
  sky_map:
    stage: sim
dir_name: Comparison



In [70]:
symlog_settings = SymLogPlotSettings(
    vmin=-1e4,
    vmax=1e4,
    linthresh=4e2,
    linscale=1.8,
    unit=None,
    ticks=[-1e4, -1e3, -4e2, -3e2, -2e2, -1e2, 0, 1e2, 2e2, 3e2, 4e2, 1e3, 1e4]
)

for freq in tqdm([30,44,70,100,143,217,353]):
    plot_differences(freq,
                     symlog_settings, 
                     out_asset=map_fig_asset,
                     show=show_figures)  # LFI freqs take ~20s, HFI freqs take ~60s

  0%|          | 0/7 [00:00<?, ?it/s]

Observation: -1.97e+03 uK_CMB - 2.15e+05 uK_CMB, Reconstruction: -3.99e+02 uK_CMB - 9.48e+04 uK_CMB, Residual: -1.21e+05 uK_CMB - 5.03e+04 uK_CMB


 14%|█▍        | 1/7 [00:06<00:40,  6.68s/it]

Saved figure to /data/jim/CMB_Data/Datasets2/DemoNotebook_full/Comparison/0/fig_maps_30.png
Observation: -5.19e+02 uK_CMB - 1.15e+05 uK_CMB, Reconstruction: -4.51e+02 uK_CMB - 4.28e+04 uK_CMB, Residual: -7.38e+04 uK_CMB - 2.44e+04 uK_CMB


 29%|██▊       | 2/7 [00:14<00:36,  7.25s/it]

...

Saved figure to /data/jim/CMB_Data/Datasets2/DemoNotebook_full/Comparison/0/fig_maps_143.png
Observation: -4.66e+02 uK_CMB - 4.74e+05 uK_CMB, Reconstruction: -3.12e+02 uK_CMB - 7.44e+04 uK_CMB, Residual: -4.00e+05 uK_CMB - 2.15e+04 uK_CMB


 86%|████████▌ | 6/7 [02:09<00:28, 28.89s/it]

Saved figure to /data/jim/CMB_Data/Datasets2/DemoNotebook_full/Comparison/0/fig_maps_217.png
Observation: -4.30e+02 uK_CMB - 3.77e+06 uK_CMB, Reconstruction: 5.51e+02 uK_CMB - 5.36e+05 uK_CMB, Residual: -3.24e+06 uK_CMB - 1.76e+05 uK_CMB


100%|██████████| 7/7 [02:43<00:00, 23.40s/it]

Saved figure to /data/jim/CMB_Data/Datasets2/DemoNotebook_full/Comparison/0/fig_maps_353.png


In [71]:
symlog_settings = SymLogPlotSettings(
    vmin=-1e7,
    vmax=1e7,
    linthresh=4000,
    linscale=0.1,
    unit=None,
    ticks=[-1e7, -1e6, -1e5, -1e4, 0, 1e4, 1e5, 1e6, 1e7]
)
for freq in tqdm([353, 545, 857]):
    plot_differences(freq, 
                     symlog_settings, 
                     out_asset=map_fig_asset,
                     show=show_figures)  # LFI freqs take ~20s, HFI freqs take ~60s
    make_hist(freq, out_asset=map_hist_asset, show=show_figures)  # ~20s

  0%|          | 0/3 [00:00<?, ?it/s]

Observation: -4.30e+02 uK_CMB - 3.77e+06 uK_CMB, Reconstruction: 5.51e+02 uK_CMB - 5.36e+05 uK_CMB, Residual: -3.24e+06 uK_CMB - 1.76e+05 uK_CMB
Saved figure to /data/jim/CMB_Data/Datasets2/DemoNotebook_full/Comparison/0/fig_maps_353.png


 33%|███▎      | 1/3 [00:23<00:46, 23.36s/it]

Saved figure to /data/jim/CMB_Data/Datasets2/DemoNotebook_full/Comparison/0/fig_hist_353.png
Observation: 4.03e+03 uK_CMB - 1.03e+08 uK_CMB, Reconstruction: 1.34e+04 uK_CMB - 1.04e+07 uK_CMB, Residual: -9.31e+07 uK_CMB - 2.79e+06 uK_CMB
Saved figure to /data/jim/CMB_Data/Datasets2/DemoNotebook_full/Comparison/0/fig_maps_545.png


 67%|██████▋   | 2/3 [00:50<00:25, 25.68s/it]

Saved figure to /data/jim/CMB_Data/Datasets2/DemoNotebook_full/Comparison/0/fig_hist_545.png
Observation: 3.80e+05 uK_CMB - 1.62e+10 uK_CMB, Reconstruction: 1.35e+06 uK_CMB - 1.24e+09 uK_CMB, Residual: -1.50e+10 uK_CMB - 1.66e+08 uK_CMB
Saved figure to /data/jim/CMB_Data/Datasets2/DemoNotebook_full/Comparison/0/fig_maps_857.png


100%|██████████| 3/3 [01:15<00:00, 25.17s/it]

Saved figure to /data/jim/CMB_Data/Datasets2/DemoNotebook_full/Comparison/0/fig_hist_857.png


In the end, I decided to simply use a couple different settings. It's not in the code, and I've manually renamed files. <!--Dreams do tend to fade, don't they?-->

In [72]:
symlog_settings = SymLogPlotSettings(
    vmin=-1e9,
    vmax=1e9,
    linthresh=1e4,
    linscale=0.5,
    unit=None,
    ticks=[-1e9, -1e8, -1e7, -1e6, -1e5, -1e4, 0, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9]
)
for freq in tqdm([857]):
    plot_differences(freq, 
                     symlog_settings, 
                     out_asset=map_fig_asset,
                     show=show_figures)  # LFI freqs take ~20s, HFI freqs take ~60s

  0%|          | 0/1 [00:00<?, ?it/s]

Observation: 3.80e+05 uK_CMB - 1.62e+10 uK_CMB, Reconstruction: 1.35e+06 uK_CMB - 1.24e+09 uK_CMB, Residual: -1.50e+10 uK_CMB - 1.66e+08 uK_CMB


100%|██████████| 1/1 [00:11<00:00, 11.75s/it]

Saved figure to /data/jim/CMB_Data/Datasets2/DemoNotebook_full/Comparison/0/fig_maps_857.png


### Graveyard of Code Cells

These are left here just to say: I ran this over and over to iron out wrinkle and check results. I then ran the previous code cells to ensure that the results were absolutely consistent.

In [73]:
# plot_differences(30,
#                  symlog_settings, 
#                  out_asset=map_fig_asset,
#                  show=show_figures)  # LFI freqs take ~20s, HFI freqs take ~60s

In [74]:
# plot_differences(44,
#                  symlog_settings, 
#                  out_asset=map_fig_asset,
#                  show=show_figures)  # LFI freqs take ~20s, HFI freqs take ~60s

In [75]:
# plot_differences(70,
#                  symlog_settings, 
#                  out_asset=map_fig_asset,
#                  show=show_figures)  # LFI freqs take ~20s, HFI freqs take ~60s

In [76]:
# plot_differences(100,
#                  symlog_settings, 
#                  out_asset=map_fig_asset,
#                  show=show_figures)  # LFI freqs take ~20s, HFI freqs take ~60s

In [77]:
# plot_differences(143,
#                  symlog_settings, 
#                  out_asset=map_fig_asset,
#                  show=show_figures)  # LFI freqs take ~20s, HFI freqs take ~60s

In [78]:
# plot_differences(217,
#                  symlog_settings, 
#                  out_asset=map_fig_asset,
#                  show=show_figures)  # LFI freqs take ~20s, HFI freqs take ~60s

In [79]:
# symlog_settings = SymLogPlotSettings(
#     vmin=-1e7,
#     vmax=1e7,
#     linthresh=4000,
#     linscale=0.1,
#     unit=None,
#     ticks=[-1e7, -1e6, -1e5, -1e4, 0, 1e4, 1e5, 1e6, 1e7]
# )

In [80]:
# plot_differences(353, 
#                  symlog_settings, 
#                  out_asset=map_fig_asset,
#                  show=show_figures)  # LFI freqs take ~20s, HFI freqs take ~60s

In [81]:
# make_hist(353, out_asset=map_hist_asset, show=show_figures)  # ~30s

In [82]:
# plot_differences(545, 
#                  symlog_settings, 
#                  out_asset=map_fig_asset,
#                  show=show_figures)  # LFI freqs take ~20s, HFI freqs take ~60s

In [83]:
# make_hist(545, out_asset=map_hist_asset, show=show_figures)  # ~20s

In [84]:
# plot_differences(857, 
#                  symlog_settings, 
#                  out_asset=map_fig_asset,
#                  show=show_figures)  # LFI freqs take ~20s, HFI freqs take ~60s

In [ ]:
# make_hist(857, out_asset=map_hist_asset, show=show_figures)  # ~20s

# Conclusion

This notebook isn't nearly as polished as some of the other tutorials. I hope it gives good insight into how I work. If nothing else, perhaps you'll feel better about the state of your code, seeing me churn over this stuff.